In [7]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import *
from pandas import DataFrame
import datetime


In [8]:
## Read in match dataset
df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')
df['year'] = pd.DatetimeIndex(df['match_date']).year

## Filter to only games from xxx onwards (Write to csv for checks)
df = df.query('year >= 2010')

## Read in Player Database
players = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\player_stats.csv')

## Merge game stats with player info
df['player_name'] = df['player_last_name'].astype(str) + ", " + df['player_first_name']
joined_data = df.merge(players, on=['player_name'],how='outer')

# Write to csv to check
# joined_data.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl_limited.csv')

# Aggregate 
avg_scores = joined_data.groupby(['player_team', 'player_first_name', 'player_last_name','player_name', 'age'])\
.agg({'afl_fantasy_score': 'mean'})\
.sort_values(by=['afl_fantasy_score'], ascending=False)\

pd.set_option('display.max_rows', 10)
# print(avg_scores)
avg_scores.query('age < 22').head(10)

C:\Users\anoble\AppData\Local\Temp\ipykernel_27952\3733721302.py:2: DtypeWarning: Columns (8,18,24,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')


afl_fantasy_score
player_team      player_first_name player_last_name player_name     age                     
Sydney           Errol             Gulden           Gulden, Errol   21.55          90.328358
Geelong          Tanner            Bruhn            Bruhn, Tanner   21.69          70.684211
                 Max               Holmes           Holmes, Max     21.44          63.333333
Adelaide         Sam               Berry            Berry, Sam      21.98          62.666667
Collingwood      Caleb             Poulter          Poulter, Caleb  21.32          62.250000
Melbourne        Jake              Bowey            Bowey, Jake     21.40          61.361702
North Melbourne  Tom               Powell           Powell, Tom     21.93          59.866667
Essendon         Archie            Perkins          Perkins, Archie 21.86          57.967742
Western Bulldogs Caleb             Poulter          Poulter, Caleb  21.32          56.666667
Geelong          Oliver            Henry            Henry, Oliver   21.52          55.863636

# Data cleaning (Priyanshu). 
Regular season games only (exclude finals)
Exclude TOG <50%
Scale up 2020 numbers by 1.2x



In [9]:
# Identify the seasons played for each player. 
# Remove seasons where game played <=5 
# Rank seasons for each player - e.g. most recent season with >5 games = 1, then 2 and so on. 
# It will be different depending on player, for example 2021 for Zac Williams will be Season 2 whereas for Jack Steele thats season 3



# Games player per year. Filter to only where games >= 5, and remove 2023 as we have no response variable for recent scores
player_summ = joined_data.groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_played=('year', 'mean'))\
    .query('matches >=5' )\

player_summ

matches  year_played
player_name       year                        
Aarts, Jake       2020.0       14       2020.0
                  2021.0       21       2021.0
                  2022.0        7       2022.0
Ablett, Gary      2010.0       24       2010.0
                  2011.0       20       2011.0
...                           ...          ...
van Rooyen, Jacob 2023.0       20       2023.0
vandenBerg, Aaron 2015.0       14       2015.0
                  2016.0       14       2016.0
                  2018.0        7       2018.0
                  2020.0       12       2020.0

[7628 rows x 2 columns]

# Seasons active factor 

In [10]:
seasons_active = player_summ.groupby(['player_name']).agg(years_active=('year_played', 'count'))

seasons_active

,years_active
player_name,
"Aarts, Jake",3
"Ablett, Gary",11
"Acres, Blake",9
"Adams, Leigh",5
"Adams, Marcus",6
...,...
"de Boer, Matt",12
"van Berlo, Jay",2
"van Berlo, Nathan",5


## In game stat factors

In [11]:
#Explode player data by years so that each year matches to every other active year
player_summ_exp = player_summ.merge(joined_data, on =['player_name'], how='inner')\
                

## Limit to games in the same year for 1yr factors, as well as 3 years up to the predicted season, and 5 years up to predicted season
player_summ_exp_1yr = player_summ_exp.query(('year_played == year'))
player_summ_exp_3yr = player_summ_exp.query(('year_played < year + 3')).query(('year_played >= year'))
player_summ_exp_5yr = player_summ_exp.query(('year_played < year + 5')).query(('year_played >= year'))

print(player_summ_exp.shape)
print(player_summ_exp_1yr.shape)
print(player_summ_exp_3yr.shape)
print(player_summ_exp_5yr.shape)

## Test matching works - should see 
pd.set_option('display.max_rows', 50)
player_summ_exp_3yr.groupby(['year_played','year']).agg(countall=('player_name', 'count')).head(50)

(984506, 92)
(123352, 92)
(303675, 92)
(423026, 92)


countall
year_played year            
2010.0      2010.0      8055
2011.0      2010.0      7088
            2011.0      8467
2012.0      2010.0      6357
            2011.0      7377
            2012.0      8940
2013.0      2011.0      6844
            2012.0      7852
            2013.0      8955
2014.0      2012.0      6994
            2013.0      7770
            2014.0      8950
2015.0      2013.0      6879
            2014.0      7834
            2015.0      8919
2016.0      2014.0      6672
            2015.0      7433
            2016.0      8913
2017.0      2015.0      6920
            2016.0      7714
            2017.0      8944
2018.0      2016.0      6755
            2017.0      7658
            2018.0      8926
2019.0      2017.0      7039
            2018.0      7783
            2019.0      8982
2020.0      2018.0      6736
            2019.0      7695
            2020.0      6915
2021.0      2019.0      7279
            2020.0      6216
            2021.0      9366
2022.0      2020.0      5622
            2021.0      8147
            2022.0      9322
2023.0      2021.0      7450
            2022.0      8209
            2023.0      9698

In [12]:
## Average factors over 1 year of game data
model_factors_agg_1yr = player_summ_exp_1yr.groupby(['player_name', 'year_played'])\
.agg({'player_height_cm': 'mean',
'player_weight_kg': 'mean',
'age': 'mean',
'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean'
})

model_factors_agg_1yr.columns = [str(col) + '_1yr' for col in model_factors_agg_1yr.columns]

model_factors_agg_1yr

player_height_cm_1yr  player_weight_kg_1yr  \
player_name       year_played                                               
Aarts, Jake       2020.0                      177.0                  76.0   
                  2021.0                      177.0                  76.0   
                  2022.0                      177.0                  76.0   
Ablett, Gary      2010.0                      182.0                  87.0   
                  2011.0                      182.0                  87.0   
...                                             ...                   ...   
van Rooyen, Jacob 2023.0                      193.0                  96.0   
vandenBerg, Aaron 2015.0                      188.0                  94.0   
                  2016.0                      188.0                  94.0   
                  2018.0                      188.0                  94.0   
                  2020.0                      188.0                  94.0   

                               age_1yr  kicks_1yr  marks_1yr  handballs_1yr  \
player_name       year_played                                                 
Aarts, Jake       2020.0         29.16   5.071429   2.214286       4.571429   
                  2021.0         29.16   5.809524   2.523810       4.761905   
                  2022.0         29.16   2.857143   0.857143       4.142857   
Ablett, Gary      2010.0         39.74  14.083333   4.416667      17.416667   
                  2011.0         39.74  15.200000   2.250000      15.050000   
...                                ...        ...        ...            ...   
van Rooyen, Jacob 2023.0           NaN   4.500000   3.100000       4.200000   
vandenBerg, Aaron 2015.0           NaN  10.214286   3.142857       6.571429   
                  2016.0           NaN   5.714286   3.071429       8.928571   
                  2018.0           NaN   8.714286   4.285714       7.142857   
                  2020.0           NaN   6.083333   2.833333       5.750000   

                               disposals_1yr  effective_disposals_1yr  \
player_name       year_played                                           
Aarts, Jake       2020.0            9.642857                 7.071429   
                  2021.0           10.571429                 8.000000   
                  2022.0            7.000000                 4.857143   
Ablett, Gary      2010.0           31.500000                      NaN   
                  2011.0           30.250000                      NaN   
...                                      ...                      ...   
van Rooyen, Jacob 2023.0            8.700000                 6.800000   
vandenBerg, Aaron 2015.0           16.785714                 9.785714   
                  2016.0           14.642857                 9.785714   
                  2018.0           15.857143                10.857143   
                  2020.0           11.833333                 7.666667   

                               disposal_efficiency_percentage_1yr  goals_1yr  \
player_name       year_played                                                  
Aarts, Jake       2020.0                                71.928571   1.000000   
                  2021.0                                69.809524   0.857143   
                  2022.0                                67.000000   0.285714   
Ablett, Gary      2010.0                                      NaN   1.833333   
                  2011.0                                      NaN   0.900000   
...                                                           ...        ...   
van Rooyen, Jacob 2023.0                                78.800000   1.400000   
vandenBerg, Aaron 2015.0                                56.000000   0.357143   
                  2016.0                                66.785714   0.857143   
                  2018.0                                69.571429   0.714286   
                  2020.0                                66.000000   0.083333   

          

In [13]:
# Build 3 Year Player Metrics
model_factors_agg_3yr = player_summ_exp_3yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean'
})

model_factors_agg_3yr.columns = [str(col) + '_3yr' for col in model_factors_agg_3yr.columns]

model_factors_agg_3yr

kicks_3yr  marks_3yr  handballs_3yr  \
player_name       year_played                                        
Aarts, Jake       2020.0        5.071429   2.214286       4.571429   
                  2021.0        5.514286   2.400000       4.685714   
                  2022.0        5.071429   2.142857       4.595238   
Ablett, Gary      2010.0       14.083333   4.416667      17.416667   
                  2011.0       14.590909   3.431818      16.340909   
...                                  ...        ...            ...   
van Rooyen, Jacob 2023.0        4.500000   3.100000       4.200000   
vandenBerg, Aaron 2015.0       10.214286   3.142857       6.571429   
                  2016.0        7.964286   3.107143       7.750000   
                  2018.0        6.714286   3.476190       8.333333   
                  2020.0        7.052632   3.368421       6.263158   

                               disposals_3yr  effective_disposals_3yr  \
player_name       year_played                                           
Aarts, Jake       2020.0            9.642857                 7.071429   
                  2021.0           10.200000                 7.628571   
                  2022.0            9.666667                 7.166667   
Ablett, Gary      2010.0           31.500000                      NaN   
                  2011.0           30.931818                      NaN   
...                                      ...                      ...   
van Rooyen, Jacob 2023.0            8.700000                 6.800000   
vandenBerg, Aaron 2015.0           16.785714                 9.785714   
                  2016.0           15.714286                 9.785714   
                  2018.0           15.047619                10.142857   
                  2020.0           13.315789                 8.842105   

                               disposal_efficiency_percentage_3yr  goals_3yr  \
player_name       year_played                                                  
Aarts, Jake       2020.0                                71.928571   1.000000   
                  2021.0                                70.657143   0.914286   
                  2022.0                                70.047619   0.809524   
Ablett, Gary      2010.0                                      NaN   1.833333   
                  2011.0                                      NaN   1.409091   
...                                                           ...        ...   
van Rooyen, Jacob 2023.0                                78.800000   1.400000   
vandenBerg, Aaron 2015.0                                56.000000   0.357143   
                  2016.0                                61.392857   0.607143   
                  2018.0                                67.714286   0.809524   
                  2020.0                                67.315789   0.315789   

                               behinds_3yr  hitouts_3yr  tackles_3yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2020.0          0.357143     0.000000     2.357143  ...   
                  2021.0          0.485714     0.000000     2.457143  ...   
                  2022.0          0.452381     0.000000     2.380952  ...   
Ablett, Gary      2010.0          1.083333     0.000000     4.208333  ...   
                  2011.0          0.977273     0.000000     5.000000  ...   
...                                    ...          ...          ...  ...   
van Rooyen, Jacob 2023.0          0.450000     3.700000     2.150000  ...   
vandenBerg, Aaron 2015.0          0.857143     0.071429     5.000000  ...   
                  2016.0          0.750000     0.071429     4.285714  ...   
                  2018.0          0.714286     0.047619     3.428571  ...   
                  2020.0          0.368421     0.000000     2.789474  ...   

                               hitout_win_percentage_3yr  intercept_marks_3yr  \
player_name       year_played                 

In [14]:
# Build 5 Year Player Metrics

model_factors_agg_5yr = player_summ_exp_5yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean'
})

model_factors_agg_5yr.columns = [str(col) + '_5yr' for col in model_factors_agg_5yr.columns]

model_factors_agg_5yr

kicks_5yr  marks_5yr  handballs_5yr  \
player_name       year_played                                        
Aarts, Jake       2020.0        5.071429   2.214286       4.571429   
                  2021.0        5.514286   2.400000       4.685714   
                  2022.0        5.071429   2.142857       4.595238   
Ablett, Gary      2010.0       14.083333   4.416667      17.416667   
                  2011.0       14.590909   3.431818      16.340909   
...                                  ...        ...            ...   
van Rooyen, Jacob 2023.0        4.500000   3.100000       4.200000   
vandenBerg, Aaron 2015.0       10.214286   3.142857       6.571429   
                  2016.0        7.964286   3.107143       7.750000   
                  2018.0        8.114286   3.342857       7.628571   
                  2020.0        6.484848   3.242424       7.393939   

                               disposals_5yr  effective_disposals_5yr  \
player_name       year_played                                           
Aarts, Jake       2020.0            9.642857                 7.071429   
                  2021.0           10.200000                 7.628571   
                  2022.0            9.666667                 7.166667   
Ablett, Gary      2010.0           31.500000                      NaN   
                  2011.0           30.931818                      NaN   
...                                      ...                      ...   
van Rooyen, Jacob 2023.0            8.700000                 6.800000   
vandenBerg, Aaron 2015.0           16.785714                 9.785714   
                  2016.0           15.714286                 9.785714   
                  2018.0           15.742857                10.000000   
                  2020.0           13.878788                 9.242424   

                               disposal_efficiency_percentage_5yr  goals_5yr  \
player_name       year_played                                                  
Aarts, Jake       2020.0                                71.928571   1.000000   
                  2021.0                                70.657143   0.914286   
                  2022.0                                70.047619   0.809524   
Ablett, Gary      2010.0                                      NaN   1.833333   
                  2011.0                                      NaN   1.409091   
...                                                           ...        ...   
van Rooyen, Jacob 2023.0                                78.800000   1.400000   
vandenBerg, Aaron 2015.0                                56.000000   0.357143   
                  2016.0                                61.392857   0.607143   
                  2018.0                                63.028571   0.628571   
                  2020.0                                67.090909   0.545455   

                               behinds_5yr  hitouts_5yr  tackles_5yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2020.0          0.357143     0.000000     2.357143  ...   
                  2021.0          0.485714     0.000000     2.457143  ...   
                  2022.0          0.452381     0.000000     2.380952  ...   
Ablett, Gary      2010.0          1.083333     0.000000     4.208333  ...   
                  2011.0          0.977273     0.000000     5.000000  ...   
...                                    ...          ...          ...  ...   
van Rooyen, Jacob 2023.0          0.450000     3.700000     2.150000  ...   
vandenBerg, Aaron 2015.0          0.857143     0.071429     5.000000  ...   
                  2016.0          0.750000     0.071429     4.285714  ...   
                  2018.0          0.771429     0.057143     4.057143  ...   
                  2020.0          0.484848     0.030303     3.121212  ...   

                               hitout_win_percentage_5yr  intercept_marks_5yr  \
player_name       year_played                 

In [15]:
# Most common position for the year
# Calculate most common position played over past year
# Consider grouping if needed to remove specificity (e.g. Left and Right indicators probably not useful)


# Games player per year. Filter to only where games >= 5
# position_mapp = joined_data.groupby(['player_name','year']).agg({'match_date': 'count'})\
#     .query('match_date >=5')

position_data = joined_data

position_data['position_grouped'] = np.select(
    [
     position_data['player_position'].isin(["FB"]), 
     position_data['player_position'].isin(["BPL", "BPR"]),
     position_data['player_position'].isin(["CHB"]), 
     position_data['player_position'].isin(["HBFL", "HBFR"]),
     position_data['player_position'].isin(["RK"]), 
     position_data['player_position'].isin(["C", "RR" ,"R"]),
     position_data['player_position'].isin(["WL", "WR"]), 
     position_data['player_position'].isin(["FF"]),
     position_data['player_position'].isin(["FPL", "FPR"]),
     position_data['player_position'].isin(["HFFL", "HFFR"]),
     position_data['player_position'].isin(["CHF"]),
     position_data['player_position'].isin(["INT", "SUB"])
    ],
    [
      'FB',
      'BP',
      'CHB',
      'HBF',
      'RUCK',
      'C',
      'W',
      'FF',
      'FP',
      'HFF',
      'CHF',
      'Ignore'  
    ], 
    default='Other'
     )

## Test no positions have not been allocated. Result = 0 rows
# joined_data.query('position_grouped in ["Other"]')\
#     .groupby(['player_position']).agg(matches=('match_date', 'count'))

position_data['matches'] = position_data.groupby(['player_name', 'year','position_grouped'])['match_id'].transform('count')

# Find most common position, ignoring sub and interchange scores
position_data['rank'] = position_data.query('position_grouped not in ["Ignore"]').groupby(['player_name', 'year'])['matches'].rank(method="first", ascending=False)

player_pos_yearly = position_data.query('rank == 1')[['player_name', 'year','position_grouped']]

# Test output
player_pos_yearly.query('player_name == "Himmelberg, Harry"')

,player_name,year,position_grouped
102837,"Himmelberg, Harry",2016.0,FP
102842,"Himmelberg, Harry",2017.0,FP
102852,"Himmelberg, Harry",2018.0,FF
102877,"Himmelberg, Harry",2019.0,FF
102901,"Himmelberg, Harry",2020.0,FP
102929,"Himmelberg, Harry",2021.0,CHF
102941,"Himmelberg, Harry",2022.0,CHF
102975,"Himmelberg, Harry",2023.0,CHB


## Response variable

In [16]:
# Average AFL fantasy score per season
response_var = joined_data.groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_int=('year', 'mean'),avg_score=('afl_fantasy_score', 'mean'))\
    .query('matches >= 5').query('avg_score > 0')\
    .sort_values(by=['avg_score'], ascending=False)

# Subtract 1 year from AFL score so it matches to factors from last year
response_var['predicted_year'] = response_var['year_int'] - 1

response_var  

matches  year_int   avg_score  predicted_year
player_name    year                                                 
Rockliff, Tom  2014.0       18    2014.0  134.777778          2013.0
Swan, Dane     2012.0       21    2012.0  130.666667          2011.0
Mitchell, Tom  2018.0       24    2018.0  128.166667          2017.0
               2017.0       22    2017.0  127.181818          2016.0
Ablett, Gary   2012.0       20    2012.0  124.700000          2011.0
...                        ...       ...         ...             ...
Wanganeen, Tex 2022.0        5    2022.0   17.600000          2021.0
Berry, Thomas  2021.0       11    2021.0   16.909091          2020.0
Morris, Josh   2021.0        5    2021.0    8.600000          2020.0
Davis, Ben     2022.0        5    2022.0    8.400000          2021.0
Chandler, Kade 2021.0        5    2021.0    2.200000          2020.0

[7628 rows x 4 columns]

## Join all factors together


In [17]:
## Player / year combos to include
factors = player_summ.merge(seasons_active, left_on=['player_name'], right_on=['player_name'], how='inner')\
           .merge(player_pos_yearly, left_on=['player_name', 'year_played'], right_on=['player_name', 'year'], how='inner')\
           .merge(model_factors_agg_1yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_3yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_5yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(response_var, left_on=['player_name', 'year_played'], right_on=['player_name', 'predicted_year'], how='inner')\

print(factors.shape)
print(factors.head(10))
factors.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\factors.csv',mode='w+')

(5594, 166)
    player_name  matches_x  year_played  years_active    year  \
0   Aarts, Jake         14       2020.0             3  2020.0   
1   Aarts, Jake         21       2021.0             3  2021.0   
2  Ablett, Gary         24       2010.0            11  2010.0   
3  Ablett, Gary         20       2011.0            11  2011.0   
4  Ablett, Gary         20       2012.0            11  2012.0   
5  Ablett, Gary         21       2013.0            11  2013.0   
6  Ablett, Gary         15       2014.0            11  2014.0   
7  Ablett, Gary          6       2015.0            11  2015.0   
8  Ablett, Gary         14       2016.0            11  2016.0   
9  Ablett, Gary         14       2017.0            11  2017.0   

  position_grouped  player_height_cm_1yr  player_weight_kg_1yr  age_1yr  \
0              HFF                 177.0                  76.0    29.16   
1              HFF                 177.0                  76.0    29.16   
2                C                 182.0       